# Importing Libraries
in this part we will install all the necessary libraries on command prompt and then import the necessary functions from those libraries. 

In [112]:
# importing all the necessary libraries
import pandas as pd

from numpy import mean
import numpy as np
import time

# step 1: preprocessing
from sklearn.impute import SimpleImputer # import some strategic imputer to fill in any missing values using mean
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MaxAbsScaler, RobustScaler, Normalizer # scale all the values to one range to avoid any biasness (this bias is seen in mostly naive bayes and knn etc)

from sklearn.impute import KNNImputer # import some strategic imputer to fill missing values using KNN (finds the nearest neighbour and fills it with that value)

from sklearn.feature_selection import SequentialFeatureSelector, SelectKBest, f_classif, VarianceThreshold

from sklearn.decomposition import PCA

# step 2: data division
from sklearn.model_selection import train_test_split, RepeatedKFold, cross_val_score # to divide the code into train/test using a specific percentage or with/without replacement

# step 3: model
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import BaggingClassifier

# step 4: displaying accuracy
from sklearn.metrics import roc_auc_score, accuracy_score # to display the accuracy of our tree

In [113]:
# use this block to install any libraries not on the system
# !pip install pandas
# !pip install sklearn
# python -m pip install scikit-learn lightgbm xgboost catboost

# Data Loading
data shall be loaded into variables as data sets using pandas and csv readers. they will be checked to see if they are loaded properly and will be loaded as 2 sets: train and test as per given in the kaggle data

In [114]:
# lets load the training data set
train_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\train_set.csv")

# lets also check it by getting the first few rows of the data, there should be x1 - x78 and one target variable Y
train_data.head() 

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,7,0.134028,93.000000,301.000000,410.010000,0,4.000000,6.0,0.0,5.000000,46.000000,0.0,69.000000,39.000000,35.900000,0.0,96.000000,59.000000,96.000000,59.000000,76.000000,50.000000,110.000000,69.000000,110.000000,69.000000,36.000000,18.000000,100.000000,97.000000,170.000000,101.000000,170.000000,101.000000,36.700000,35.900000,61.000000,59.000000,61.000000,59.000000,61.000000,52.000000,74.000000,71.000000,74.000000,71.000000,27.000000,19.000000,100.000000,99.000000,105.000000,102.000000,105.000000,102.000000,83.000000,83.000000,4.200000,4.200000,0.110000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,4,0.203574,84.118204,260.708348,337.974019,0,4.000000,6.0,0.0,5.000000,58.437953,0.0,75.029213,54.532835,36.097079,0.0,74.656929,57.591260,74.656929,57.591260,81.715355,69.372284,98.656929,67.875905,98.656929,67.875905,35.788267,16.934331,95.313858,90.343071,137.437953,102.649685,137.437953,102.649685,36.887591,36.009488,72.248189,71.153307,72.248189,71.153307,79.744567,74.839449,96.467165,93.182520,96.467165,93.182520,19.562047,16.934331,93.751811,92.656929,135.248189,133.496378,135.248189,133.496378,139.897874,139.897874,3.718976,3.718976,0.100292,0.033431,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,7,0.116667,75.700000,302.000000,105.010000,0,4.000000,6.0,0.0,5.000000,55.000000,0.0,129.000000,42.000000,36.200000,0.0,113.000000,65.000000,113.000000,65.000000,86.000000,55.000000,137.000000,83.000000,137.000000,83.000000,31.000000,7.000000,98.000000,91.000000,163.000000,107.000000,163.000000,107.000000,37.400000,36.200000,113.000000,82.000000,113.000000,82.000000,86.000000,60.000000,137.000000,102.000000,137.000000,102.000000,30.000000,20.000000,97.000000,93.000000,163.000000,126.000000,163.000000,126.000000,109.000000,109.000000,3.800000,3.800000,0.020000,0.010000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,7,0.052654,73.956249,121.015914,413.698989,0,3.015914,6.0,0.0,5.000000,50.718754,0.0,141.665798,31.238716,36.608249,0.0,93.127315,49.222801,93.127315,49.222801,94.018521,54.270545,98.366031,66.190973,98.366031,66.190973,31.015914,15.968171,99.015914,92.079572,146.079572,100.015914,146.079572,100.015914,37.050796,36.608249,82.302373,80.968171,82.302373,80.968171,80.193579,79.970778,96.397860,92.111401,96.397860,92.111401,19.143229,18.952257,98.031829,97.984086,149.015914,132.936342,149.015914,132.936342,161.697627,85.254630,4.290451,4.285677,0.108249,0.039363,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,5,1.550794,52.311866,169.455275,136.996895,0,4.000000,6.0,0.0,4.308058,108.919416,0.0,51.232234,38.459708,36.461612,0.0,80.066306,45.540292,80.066306,45.540292,103.459708,87.383883,89.682423,58.232234,89.682423,58.232234,31.691942,18.383883,100.000000,97.000000,122.990481,91.530773,122.990481,91.530773,37.715165,36.461612,77.601839,76.677663,77.601839,76.677663,102.075825,101.459708,86.909897,86.601839,86.909897,86.601839,31.691942,30.767766,99.691942,99.075825,116.829313,114.364845,116.829313,114.364845,114.450189,109.213196,3.769194,3.769194,0.164645,0.069242,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0


In [115]:
# lets load the test data
test_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\test_set.csv")

# check if the data has been loaded by getting the first 5 rows - there should be x1 - x78 and no target variable Y as this is test data
test_data.head() 

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78
0,300001,79.000000,17.122318,0,0,1,170.200000,1,700,0,4,0.118056,49.600000,118.000000,602.120000,0,3.000000,4.000000,0.0,4.000000,134.000000,0.0,59.000000,11.000000,36.100000,0.000000,82.000000,63.000000,82.000000,63.000000,106.000000,80.000000,113.000000,89.000000,113.000000,89.000000,16.000000,16.000000,95.0,91.000000,151.000000,123.000000,151.000000,123.000000,37.600000,36.100000,80.000000,80.000000,80.000000,80.000000,104.000000,104.000000,113.000000,113.000000,113.000000,113.000000,16.000000,16.000000,95.0,95.000000,151.000000,151.000000,151.000000,151.000000,138.000000,138.000000,3.300000,3.300000,0.070000,0.030000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,300002,38.000000,43.693579,0,0,1,165.100000,1,814,0,7,0.010417,119.100000,122.000000,703.030000,0,1.000000,1.000000,0.0,1.000000,41.000000,1.0,113.000000,16.000000,36.600000,1.000000,111.000000,47.000000,111.000000,47.000000,79.000000,41.000000,125.000000,60.000000,125.000000,60.000000,31.000000,16.000000,100.0,92.000000,167.000000,99.000000,167.000000,99.000000,37.300000,36.600000,108.000000,63.000000,108.000000,63.000000,46.000000,42.000000,125.000000,84.000000,125.000000,84.000000,16.000000,16.000000,100.0,100.000000,159.000000,131.000000,159.000000,131.000000,352.000000,83.000000,4.600000,3.900000,0.050000,0.040000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,300003,36.064225,23.998944,0,0,1,167.086735,1,662,0,7,0.004261,67.592737,122.000000,703.030000,0,2.610444,5.305222,0.0,1.915666,100.116446,0.0,89.602340,12.610444,36.730522,0.694778,104.895558,58.148559,104.895558,58.148559,107.200780,82.811224,116.148559,68.064225,116.148559,68.064225,24.084334,12.915666,100.0,92.220888,150.602340,100.674669,150.602340,100.674669,37.247389,36.669478,104.895558,71.253001,104.895558,71.253001,102.200780,88.200780,116.148559,83.337335,116.148559,83.337335,18.220888,14.136555,100.0,93.610444,150.297118,117.285114,150.297118,117.285114,145.457985,93.875450,3.577911,3.577911,0.006948,0.006948,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,300004,61.846764,31.693449,0,3,1,182.355708,2,862,0,7,0.008327,105.981746,167.488094,272.921476,0,3.684676,6.000000,0.0,5.000000,130.738706,0.0,142.513736,35.775030,36.931532,0.000000,97.558913,57.189560,97.558913,57.189560,127.792735,92.414530,142.513736,71.604090,142.513736,71.604090,40.504884,14.162088,100.0,94.261294,153.748473,110.018620,153.748473,110.018620,37.920726,36.931532,78.820207,70.297619,78.820207,70.297619,105.883089,96.567765,95.550061,81.504884,95.550061,81.504884,28.099206,25.099206,100.0,98.000000,144.550061,118.586385,144.550061,118.586385,223.029302,153.875152,4.216209,4.010806,0.062613,0.033153,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,300005,71.591991,20.086147,1,0,1,166.704917,2,335,0,6,0.000000,56.072948,302.514572,1257.439179,1,4.000000,6.000000,0.0,5.000000,49.689433,0.0,80.039155,23.533704,35.120400,0.000000,82.087430,60.029143,82.087430,60.029143,75.485428,51.204004,104.514572,80.883426,104.514572,80.883426,19.883426,10.485428,100.0,94.970857,176.397998,131.058287,176.397998,131.058287,37.042714,34.914572,69.806007,64.660289,69.806007,64.660289,60.631146,54.233148,90.145717,85.514572,90.145717,85.514572,17.825139,11.941713,100.0,98.514572,140.835150,133.116574,140.835150,133.116574,153.979978,105.437152,4.139800,4.139800,0.014854,0.004854,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Data Preprocessing
before we start processing this data and using algorithms, we will fix this data first, this is called data preprocessing

## Conversion of Categorical to Numerical
First we will convert categorical data to numerical data by doing one hot encoding, which turns it into binary variables

In [116]:
# one hot encoding - display it
pd.get_dummies(train_data) # this line will convert the train_data to one hot encoding but it will only display the result and not save it

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78,Y
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,7,0.134028,93.000000,301.000000,410.010000,0,4.000000,6.0,0.0,5.000000,46.000000,0.000000,69.000000,39.000000,35.900000,0.0,96.000000,59.000000,96.000000,59.000000,76.000000,50.000000,110.000000,69.000000,110.000000,69.000000,36.000000,18.000000,100.000000,97.000000,170.000000,101.000000,170.000000,101.000000,36.700000,35.900000,61.000000,59.000000,61.000000,59.000000,61.000000,52.000000,74.000000,71.000000,74.000000,71.000000,27.000000,19.000000,100.000000,99.000000,105.000000,102.000000,105.000000,102.000000,83.000000,83.000000,4.200000,4.200000,0.110000,0.040000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,4,0.203574,84.118204,260.708348,337.974019,0,4.000000,6.0,0.0,5.000000,58.437953,0.000000,75.029213,54.532835,36.097079,0.0,74.656929,57.591260,74.656929,57.591260,81.715355,69.372284,98.656929,67.875905,98.656929,67.875905,35.788267,16.934331,95.313858,90.343071,137.437953,102.649685,137.437953,102.649685,36.887591,36.009488,72.248189,71.153307,72.248189,71.153307,79.744567,74.839449,96.467165,93.182520,96.467165,93.182520,19.562047,16.934331,93.751811,92.656929,135.248189,133.496378,135.248189,133.496378,139.897874,139.897874,3.718976,3.718976,0.100292,0.033431,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,7,0.116667,75.700000,302.000000,105.010000,0,4.000000,6.0,0.0,5.000000,55.000000,0.000000,129.000000,42.000000,36.200000,0.0,113.000000,65.000000,113.000000,65.000000,86.000000,55.000000,137.000000,83.000000,137.000000,83.000000,31.000000,7.000000,98.000000,91.000000,163.000000,107.000000,163.000000,107.000000,37.400000,36.200000,113.000000,82.000000,113.000000,82.000000,86.000000,60.000000,137.000000,102.000000,137.000000,102.000000,30.000000,20.000000,97.000000,93.000000,163.000000,126.000000,163.000000,126.000000,109.000000,109.000000,3.800000,3.800000,0.020000,0.010000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,7,0.052654,73.956249,121.015914,413.698989,0,3.015914,6.0,0.0,5.000000,50.718754,0.000000,141.665798,31.238716,36.608249,0.0,93.127315,49.222801,93.127315,49.222801,94.018521,54.270545,98.366031,66.190973,98.366031,66.190973,31.015914,15.968171,99.015914,92.079572,146.079572,100.015914,146.079572,100.015914,37.050796,36.608249,82.302373,80.968171,82.302373,80.968171,80.193579,79.970778,96.397860,92.111401,96.397860,92.111401,19.143229,18.952257,98.031829,97.984086,149.015914,132.936342,149.015914,132.936342,161.697627,85.254630,4.290451,4.285677,0.108249,0.039363,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,5,1.550794,52.311866,169.455275,136.996895,0,4.000000,6.0,0.0,4.308058,108.919416,0.000000,51.232234,38.459708,36.461612,0.0,80.066306,45.540292,80.066306,45.540292,103.459708,87.383883,89.682423,58.232234,89.682423,58.232234,31.691942,18.383883,100.000000,97.000000,122.990481,91.530773,122.990481,91.530773,37.715165,36.461612,77.601839,76.677663,77.601839,76.677663,102.075825,101.459708,86.909897,86.601839,86.909897,86.601839,31.691942,30.767766,99.691942,99.075825,116.829313,114.364845,116.829313,114.364845,114.450189,109.213196,3.769194,3.769194,0.164645,0.069242,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246117,246118

In [117]:
# we can see that there is no change in the number of columns meaning there is no categorical data. but for the sake of running the program. we must perform the preprocessing therefore we shall re-run the one hot encoding and save it somewhere
train_data_processed = pd.get_dummies(train_data)

# now we shall do the same on the test data so that we maintain the rules over all data
test_data_processed = pd.get_dummies(test_data)

## Data Removal
Removal of rows that have empty cells

In [118]:
# ----------------------------- case 61a, 61b -----------------------------
# train_data_processed = train_data_processed.dropna(axis=0)
# train_data_processed.shape

## ERROR
# row dropping does not work in NaiveBayes as the test_data also contains NaN values and we cannot drop those rows. NaiveBayes then throws error that it cannot fit model due to NaN values. 

## Data Splitting - festures and targets
the data in train_data set is of x1 - x78 columns (79 variables) and one target variable (Y). we must split that data so that we can perform data preprocessing on the features variables (will be referred to as X).

In [119]:
# so in X, it is ALL the columns EXCEPT the last column known as 'Y' (we can confirm this using the train_data.head() we did earlier) so we must get all columns and DROP only the 'y' column
X = train_data_processed.drop(columns=['Y'])
X # lets display X and see what it is now

,RecordId,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,X12,X13,X14,X15,X16,X17,X18,X19,X20,X21,X22,X23,X24,X25,X26,X27,X28,X29,X30,X31,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41,X42,X43,X44,X45,X46,X47,X48,X49,X50,X51,X52,X53,X54,X55,X56,X57,X58,X59,X60,X61,X62,X63,X64,X65,X66,X67,X68,X69,X70,X71,X72,X73,X74,X75,X76,X77,X78
0,1,87.000000,34.118411,0,2,0,165.100000,1,829,2,7,0.134028,93.000000,301.000000,410.010000,0,4.000000,6.0,0.0,5.000000,46.000000,0.000000,69.000000,39.000000,35.900000,0.0,96.000000,59.000000,96.000000,59.000000,76.000000,50.000000,110.000000,69.000000,110.000000,69.000000,36.000000,18.000000,100.000000,97.000000,170.000000,101.000000,170.000000,101.000000,36.700000,35.900000,61.000000,59.000000,61.000000,59.000000,61.000000,52.000000,74.000000,71.000000,74.000000,71.000000,27.000000,19.000000,100.000000,99.000000,105.000000,102.000000,105.000000,102.000000,83.000000,83.000000,4.200000,4.200000,0.110000,0.040000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
1,2,82.372284,31.573280,0,0,1,162.983897,1,724,0,4,0.203574,84.118204,260.708348,337.974019,0,4.000000,6.0,0.0,5.000000,58.437953,0.000000,75.029213,54.532835,36.097079,0.0,74.656929,57.591260,74.656929,57.591260,81.715355,69.372284,98.656929,67.875905,98.656929,67.875905,35.788267,16.934331,95.313858,90.343071,137.437953,102.649685,137.437953,102.649685,36.887591,36.009488,72.248189,71.153307,72.248189,71.153307,79.744567,74.839449,96.467165,93.182520,96.467165,93.182520,19.562047,16.934331,93.751811,92.656929,135.248189,133.496378,135.248189,133.496378,139.897874,139.897874,3.718976,3.718976,0.100292,0.033431,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
2,3,50.000000,27.771653,0,0,1,165.100000,1,895,2,7,0.116667,75.700000,302.000000,105.010000,0,4.000000,6.0,0.0,5.000000,55.000000,0.000000,129.000000,42.000000,36.200000,0.0,113.000000,65.000000,113.000000,65.000000,86.000000,55.000000,137.000000,83.000000,137.000000,83.000000,31.000000,7.000000,98.000000,91.000000,163.000000,107.000000,163.000000,107.000000,37.400000,36.200000,113.000000,82.000000,113.000000,82.000000,86.000000,60.000000,137.000000,102.000000,137.000000,102.000000,30.000000,20.000000,97.000000,93.000000,163.000000,126.000000,163.000000,126.000000,109.000000,109.000000,3.800000,3.800000,0.020000,0.010000,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
3,4,66.236109,26.515922,0,0,1,167.009549,1,637,0,7,0.052654,73.956249,121.015914,413.698989,0,3.015914,6.0,0.0,5.000000,50.718754,0.000000,141.665798,31.238716,36.608249,0.0,93.127315,49.222801,93.127315,49.222801,94.018521,54.270545,98.366031,66.190973,98.366031,66.190973,31.015914,15.968171,99.015914,92.079572,146.079572,100.015914,146.079572,100.015914,37.050796,36.608249,82.302373,80.968171,82.302373,80.968171,80.193579,79.970778,96.397860,92.111401,96.397860,92.111401,19.143229,18.952257,98.031829,97.984086,149.015914,132.936342,149.015914,132.936342,161.697627,85.254630,4.290451,4.285677,0.108249,0.039363,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
4,5,81.303299,20.843691,0,0,1,158.165419,0,564,0,5,1.550794,52.311866,169.455275,136.996895,0,4.000000,6.0,0.0,4.308058,108.919416,0.000000,51.232234,38.459708,36.461612,0.0,80.066306,45.540292,80.066306,45.540292,103.459708,87.383883,89.682423,58.232234,89.682423,58.232234,31.691942,18.383883,100.000000,97.000000,122.990481,91.530773,122.990481,91.530773,37.715165,36.461612,77.601839,76.677663,77.601839,76.677663,102.075825,101.459708,86.909897,86.601839,86.909897,86.601839,31.691942,30.767766,99.691942,99.075825,116.829313,114.364845,116.829313,114.364845,114.450189,109.213196,3.769194,3.769194,0.164645,0.069242,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246117,246118,65.149110,33.35

In [120]:
# so as per our X output, we can see that number of columns in train_data is 79 and number of columns in X is 78 meaning we have successfully performed our removal of target variable
# now to get the target variable alone, we can just get it alone,
Y = train_data_processed['Y']
Y.shape # lets see what it is
# as per our Y output, we can see it is of one column and 246k rows which means we have successfully extracted the target variable column

(246122,)

## Data Imputation 
many cells in our data may be empty - we must fill these cells with data. we have multiple options to deal with them:
- we remove the entire rows (Case 1)
- we fill the cells with the average of the column (Case 2)
- we fill the cells based on KNN imputation (nearest neighbour) (Case 3)

In [121]:
# REMOVE ROWS 
# ----------------------------- case  -----------------------------
# in this case, lets remove the entire rows that have NaN values. before saving the removed rows data set, lets first run it and display it to see the outcome, then we shall save in X
# X.dropna(axis=0)

In [122]:
# REMOVE ROWS
# # so we originally had 246122 rows and now after removing empty cell rows we have 239650 rows which is a 6472 rows difference. as our first try, lets work with it. lets assign this data set in place of X
# X = X.dropna(axis=0)
# X
# these above 2 lines were commented out as there was an error handling, rows were being removed from X and not from Y so we fixed it by removing from train_data and then splitting into X and Y
# train_data_processed = train_data_processed.dropna(axis=0)
# train_data_processed.shape

In [123]:
# Average Mean Imputation
# ----------------------------- case 31, 32, 34 to 42, 45, 48, 49, 50, 164, 165a -----------------------------
# this will fill all the empty spaces using the average of all the spaces
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)                                        # fill them in X
test_data_processed = imputer.transform(test_data_processed)    # fill them in test data

In [124]:
# KNN Imputation
# ----------------------------- case 33 -----------------------------
# this fills them in using k-nearest neighbours of all the spaces
# imputer = KNNImputer(n_neighbors=7)
# X = imputer.fit_transform(X)                                        # fill them in X
# test_data_processed = imputer.fit_transform(test_data_processed)    # fill them in test data

## Data Scaling
some columns may be very large then other columns when compared. it would not affect at the moment as we are using decision trees, but to maintain a fair enviroment, we shall perform scaling on every run.
there are two types of scaling: 
- min max scaling (also known as normalization)
- standardisation (z-score normalization)
- max abs scaler
- robust scaler
- normalizer

In [125]:
# ----------------------------- case 32 to 42, 45, 48, 49, 50, 164, 165a -----------------------------
# in this case we shall perform min max scaling. to do that, we must use our MinMaxScaler that we have imported above
scaler = MinMaxScaler()
# now we must use this scaler to scale X
scaler.fit_transform(X)

array([[0.00000000e+00, 9.72602740e-01, 3.63856188e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [4.06304216e-06, 9.09209364e-01, 3.15807703e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [8.12608432e-06, 4.65753425e-01, 2.44038356e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [9.99991874e-01, 5.68110690e-01, 5.09895343e-01, ...,
        0.00000000e+00, 0.00000000e+00, 4.12013395e-01],
       [9.99995937e-01, 6.84931507e-01, 1.67901180e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [1.00000000e+00, 4.69282739e-01, 3.38096342e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [126]:
# ----------------------------- case 31 -----------------------------
# scaler = MaxAbsScaler()
# # now we must use this scaler to scale X
# scaler.fit_transform(X)

In [127]:
# our output shows us that every value in the array is between 0 and 1. thus lets save this value on X
X = scaler.fit_transform(X)

# now we must do the same on our test_data set
test_data_processed = scaler.transform(test_data_processed)

## Filters
there are two types of filters to filter out columns/features:
- variance filter (a column which has same values throughout the column like all are sunny)
- correlation filter (two columns which are same like weight in kg and weight in pounds)

In [128]:
print("X : ", X.shape)
print("test data : ", test_data_processed.shape)

X :  (246122, 78)
test data :  (105482, 78)


In [129]:
# variance filter
# ----------------------------- case 36 to 40d -----------------------------
# variance_filter = VarianceThreshold(threshold=0.0001) # Adjust threshold accordingly
# -- 
# variance_filter = VarianceThreshold(threshold=0.1)    # case 36
# variance_filter = VarianceThreshold(threshold=0.5)    # case 37a
# variance_filter = VarianceThreshold(threshold=0.3)    # case 37b 
# variance_filter = VarianceThreshold(threshold=0.01)   # case 37c
# variance_filter = VarianceThreshold(threshold=0.05)   # case 37d
# variance_filter = VarianceThreshold(threshold=0.03)   # case 37e. 37f
# variance_filter = VarianceThreshold(threshold=0.01)   # case 38
# variance_filter = VarianceThreshold(threshold=0.005)  # case 39a
# variance_filter = VarianceThreshold(threshold=0.001)  # case 39b
# variance_filter = VarianceThreshold(threshold=0.0001) # case 40a, 40b, 40c, 40d

In [130]:
# X = variance_filter.fit_transform(X)
# test_data_processed = variance_filter.transform(test_data_processed)
X.shape

(246122, 78)

In [131]:
test_data_processed.shape

(105482, 78)

In [132]:
# # correlation filter
# # ----------------------------- case 36 to 40d -----------------------------
# corr_matrix = pd.DataFrame(X).corr().abs()
# upper_tri = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.87)]
# --
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]    # case 36 to 37d, 37f to 40d
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.8)]    # case 37e, 40b
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.85)]   # case 40c
# to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.87)]   # case 40d

In [133]:
# X = pd.DataFrame(X).drop(columns=to_drop)
# test_data_processed = pd.DataFrame(test_data_processed).drop(columns=to_drop)
X.shape

(246122, 78)

In [134]:
test_data_processed.shape

(105482, 78)

# PCA

In [135]:
# ----------------------------- case -----------------------------
pca = PCA(n_components=24)                    
X = pca.fit_transform(X)
test_data_processed = pca.transform(test_data_processed)

# Data Splitting - train and validate
now our test_data set is of rows with NO target variable whereas the train_data set is WITH target variable.
our rules in machine learning is that we must train half or 70% of the data and then we must check its accuracy using the remaining half or 30% of the data - we can only check accuracy IF we have the answers i.e. the target variable. 
So, what we need to do is, is split the train_data set into 2, by a 70% and 30% ratio. we train the model using the 70% and then test the model using the 30% and then use that model to predict the test_data set.

In [136]:
# holdout method
trainX, testX, trainY, testY = train_test_split(X, Y, test_size=0.3)
print("trainX shape -> ", trainX.shape)
print("trainY shape -> ", trainY.shape)
print("testX shape -> ", testX.shape)
print("testY shape -> ", testY.shape)

trainX shape ->  (172285, 24)
trainY shape ->  (172285,)
testX shape ->  (73837, 24)
testY shape ->  (73837,)


# Model intialization
here model is intialized

In [137]:
model = GaussianNB()

## Feature Selection
in this we select columns and features we want to keep. there are several algos to do so:
- forward selection
- backward selection
- Kbest (best out of all)

In [138]:
# forward selection
# ----------------------------- case 34, 35a, 35b, 41a, 42, 45, 48, 49, 50, 61a, 164 -----------------------------
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=15, scoring='roc_auc')  # case 61a
# --
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=5, scoring='roc_auc')   # case 34
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=10, scoring='roc_auc')  # case 35a
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=20, scoring='roc_auc')  # case 35b
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=30, scoring='roc_auc')  # case 41a
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=25, scoring='roc_auc')  # case 42
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=15, scoring='roc_auc')  # case 45, 164
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=17, scoring='roc_auc')  # case 48
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=13, scoring='roc_auc')  # case 49
# selection = SequentialFeatureSelector(model, direction='forward',n_features_to_select=14, scoring='roc_auc')  # case 50

In [139]:
# # backward selection
# selection = SequentialFeatureSelector(model, direction='backward',n_features_to_select=5, scoring='roc_auc', n_jobs=-1)

In [140]:
# k best
# ----------------------------- case 41b -----------------------------
# selection = SelectKBest(score_func=f_classif, k=30)             # Use f_classif for classification

In [141]:
# feature selection fitting
# trainX = selection.fit_transform(trainX, trainY)

In [142]:
# feature selection applying
# testX = selection.transform(testX)                                  # Ensure the test set is transformed similarly
# test_data_processed = selection.transform(test_data_processed)      # test data is also transformed
# X = selection.transform(X)                                          # full data transforming

In [143]:
trainX.shape

(172285, 24)

# feature importance

In [144]:
# # ----------------------- case 165a ----------------------
# features = 20
# print("fitting")
    
# # fit the model
# model.fit(trainX, trainY)

# print("extracting features")

# # extract all the feature names from data
# importances = model.feature_importances_
# feature_names = train_data_processed.drop(columns=['Y']).columns
# print(feature_names)

# # sort with respect to importance
# feature_importance_df = pd.DataFrame({
#     'Feature': feature_names,
#     'Importance': importances
# }).sort_values(by='Importance', ascending=False)

# # extract the top ones
# top_features = feature_importance_df['Feature'].head(features).values
# print(top_features)

# # change all data according to the top ones we have selected
# trainX = pd.DataFrame(trainX, columns=feature_names)[top_features]
# testX = pd.DataFrame(testX, columns=feature_names)[top_features]
# X = pd.DataFrame(X, columns=feature_names)[top_features]
# test_data_processed = pd.DataFrame(test_data_processed, columns=feature_names)[top_features]

# print("features extracted")
    
# # retrain the model
# model.fit(trainX, trainY)

# print("features trained")

# Bagging

In [145]:
# ------------------------ case 164 -----------------------
# model = BaggingClassifier(estimator=model, n_estimators=50, verbose=2)

# Model running
here we run the model

In [146]:
model.fit(trainX, trainY)

y_pred = model.predict(testX)

accuracy = accuracy_score(testY, y_pred)
print("model accuracy = ", accuracy, "   ")

# now lets calculate the ROC AUC score according to this prediction
roc_score = roc_auc_score(testY, y_pred)
print("roc score = ", roc_score, "   ")

model accuracy =  0.9702181832956377    
roc score =  0.6879233888954087    


## predict for test dataset
fit the model and predict for test dataset

In [147]:
model.fit(X, Y)

test_prediction = model.predict_proba(test_data_processed)

test_prediction=test_prediction[:, 1]

print(test_prediction)

[1.59211514e-04 3.53841249e-01 4.40780800e-05 ... 1.98533092e-05
 7.90237187e-04 1.66774868e-04]


## write into csv
now we write the predictions into the csv file

In [148]:
sample_data = pd.read_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\sample_submission.csv")

sample_data['Y'] = test_prediction
sample_data

sample_data.to_csv(r"D:\Users\DELL\OneDrive - Institute of Business Administration\IBA\sem5\machine learning\ipynb notebooks\challenger1\iml-fall-2024-challenge-1\nb1.csv", index=False)
sample_data

,RecordId,Y
0,300001,0.000159
1,300002,0.353841
2,300003,0.000044
3,300004,0.000012
4,300005,0.000012
...,...,...
105477,405478,0.000007
105478,405479,0.992244
105479,405480,0.000020
105480,405481,0.000790
